# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
from pymystem3 import Mystem
import re
import time
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
from catboost import CatBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn import svm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


*Кратно ознакомимся с данными*

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
df.isna().sum()

text     0
toxic    0
dtype: int64

In [5]:
df.duplicated().sum()

0

In [6]:
df.describe()

,toxic
count,159571.000000
mean,0.101679
std,0.302226
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [7]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Очистим тексты

In [8]:
def clear_text(text):
    cl_text = re.sub(r"[^a-zA-Z' ]", ' ', text)
    cl_text = cl_text.split()
    return ' '.join(cl_text)

Лемматизируем наши тексты

In [9]:
def lemmatize(text):
    text = clear_text(text)
    word_list = nltk.word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(text) for text in word_list])

In [10]:
df['lemma'] = df['text'].apply(lambda x: clear_text(lemmatize(x)))

Поделим датасет на train и test выборки

In [11]:
train, test = train_test_split(df, shuffle=True, test_size=0.2, random_state=12345)

Избавимся от "лишних слов"

Векторизируем корпуса

In [12]:
corpus = train['lemma']
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_features_train = TfidfVectorizer(stop_words=stopwords)

features_train = count_features_train.fit_transform(corpus) 

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
features_test = count_features_train.transform(test['lemma'])

target_train = train['toxic']
target_test = test['toxic']

In [16]:
print(f'Размер train корпуса {features_train.shape}')
print(f'Размер test корпуса {features_test.shape}')

Размер train корпуса (127656, 144122)
Размер test корпуса (31915, 144122)


#### Вывод

Данные представлены, изучены, лемматиризованы и очищены от стоп-слов  
Подготовлены обучающая и тестовая выборка в соотношении 1/5

## Обучение

Сперва обучим **логистическую регрессию**

In [17]:
%%time
lr_model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced', max_iter=1000)
lr_model.fit(features_train, target_train)
predictions_lr = lr_model.predict(features_test)
accuracy_lr = accuracy_score(target_test, predictions_lr)

CPU times: user 9.45 s, sys: 6.8 s, total: 16.2 s
Wall time: 16.2 s


In [18]:
f1_lr_model = f1_score(target_test, predictions_lr)

In [19]:
print(f'F1 score на тестовой выборке: {f1_lr_model}')

F1 score на тестовой выборке: 0.7468644639244774


Теоретически, можно округлить результат и получим искомый, но, пойдем копать глубже

In [23]:
%%time
params = {'C': list(range(1,15,3)),
          'max_iter': [1000]}
lr_model_second = LogisticRegression()
clf = GridSearchCV(lr_model_second, params, cv=5, scoring='f1', n_jobs=-1, verbose=2)
clf.fit(features_train, target_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] C=1, max_iter=1000 ..............................................


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............................... C=1, max_iter=1000, total=   9.7s
[CV] C=1, max_iter=1000 ..............................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    9.7s remaining:    0.0s


[CV] ............................... C=1, max_iter=1000, total=  11.5s
[CV] C=1, max_iter=1000 ..............................................
[CV] ............................... C=1, max_iter=1000, total=   8.3s
[CV] C=1, max_iter=1000 ..............................................
[CV] ............................... C=1, max_iter=1000, total=   8.1s
[CV] C=1, max_iter=1000 ..............................................
[CV] ............................... C=1, max_iter=1000, total=   9.1s
[CV] C=4, max_iter=1000 ..............................................
[CV] ............................... C=4, max_iter=1000, total=  13.4s
[CV] C=4, max_iter=1000 ..............................................
[CV] ............................... C=4, max_iter=1000, total=  16.4s
[CV] C=4, max_iter=1000 ..............................................
[CV] ............................... C=4, max_iter=1000, total=  14.2s
[CV] C=4, max_iter=1000 ..............................................
[CV] .

[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  6.1min finished


CPU times: user 3min 33s, sys: 2min 51s, total: 6min 25s
Wall time: 6min 26s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [1, 4, 7, 10, 13], 'max_iter': [1000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=2)

In [25]:
%%time
lr_model_second = LogisticRegression(C=10, max_iter=1000)
lr_model_second.fit(features_train, target_train)
predictions_lr_second = lr_model_second.predict(features_test)
f1_lr_model_second = f1_score(target_test, predictions_lr_second)

CPU times: user 8.81 s, sys: 5.95 s, total: 14.8 s
Wall time: 14.8 s


In [26]:
print(f'F1 score на тестовой выборке: {f1_lr_model_second}')

F1 score на тестовой выборке: 0.7781605235962796


Попадание в целевой показатель

**Случайный лес**

In [27]:
%%time
rfc_model = RandomForestClassifier(class_weight='balanced', max_depth=20, n_estimators=50)
rfc_model.fit(features_train, target_train)
predictions_rfc = rfc_model.predict(features_test)
f1_rfc_model = f1_score(target_test, predictions_rfc)

CPU times: user 7.17 s, sys: 0 ns, total: 7.17 s
Wall time: 7.18 s


In [28]:
print(f'F1 score на тестовой выборке: {f1_rfc_model}')

F1 score на тестовой выборке: 0.3758716875871688


**Stochastic Gradient Descent Classifier**

In [29]:
%%time
sgd_model = SGDClassifier(shuffle=True, random_state=12345)
sgd_model.fit(features_train, target_train)
predictions_sgd = sgd_model.predict(features_test)
f1_sgd_model = f1_score(target_test, predictions_sgd)

CPU times: user 503 ms, sys: 0 ns, total: 503 ms
Wall time: 520 ms


In [30]:
print(f'F1 score на тестовой выборке: {f1_sgd_model}')

F1 score на тестовой выборке: 0.6241905159807813


#### Вывод

Модели обучены, метрики получены

## Выводы

Наилучший результат показала модель логистической регрессии, **0.7781605235962796**  
Другие модели глубоко с гиперпараметрами не играл, т.к. очень медленное время выполнения. У ряда моделей, типа CatBoost и т.п., вообще падает ядро...  
С BERT поиграю в праздники, интересно, какой результат можно выжать из нее...